# Visualize results

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.1.1
qiskit-ibm-runtime~=0.40.1
```
</details>

## Plot histogram <a name='histogram'></a>

The `plot_histogram` function visualizes the result of sampling a quantum circuit on a QPU.

<Admonition title="Using the output from functions" type="tip">
This function returns a `matplotlib.Figure` object. When the last line of a code cell outputs these objects, Jupyter notebooks display them below the cell. If you call these functions in some other environments or in scripts, you will need to explicitly show or save the outputs.

Two options are:
- Call `.show()` on the returned object to open the image in a new window (assuming your configured matplotlib backend is interactive).
- Call `.savefig("out.png")` to save the figure to `out.png` in the current working directory. The `savefig()` method takes a path so you can adjust the location and filename where you're saving the output. For example, `plot_state_city(psi).savefig("out.png")`.
</Admonition>

For example, make a two-qubit Bell state:

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler import generate_preset_pass_manager

from qiskit.circuit import QuantumCircuit
from qiskit.visualization import plot_histogram

service = QiskitRuntimeService()

backend = service.least_busy(simulator=False, operational=True)

In [2]:
# Quantum circuit to make a Bell state
bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(bell)

# execute the quantum circuit
sampler = Sampler(backend)
job = sampler.run([isa_circuit])
result = job.result()

print(result)

PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': {'__type__': 'ExecutionSpans', '__value__': {'spans': [DoubleSliceSpan(<start='2025-09-11 15:56:43', stop='2025-09-11 15:56:44', size=4096>)]}}}, 'version': 2})


In [3]:
plot_histogram(result[0].data.meas.get_counts())

<Image src="/docs/images/guides/visualize-results/extracted-outputs/57d8053e-d030-460d-9c1f-772e53b1a49b-0.avif" alt="Output of the previous code cell" />

### Options when plotting a histogram

Use the following options for `plot_histogram` to adjust the output graph.

* `legend`: Provides a label for the executions. It takes a list of strings used to label each execution's results. This is mostly useful when plotting multiple execution results in the same histogram
* `sort`: Adjusts the order of the bars in the histogram. It can be set to either ascending order with `asc` or descending order with `desc`
* `number_to_keep`: Takes an integer for the number of terms to show. The rest are grouped together in a single bar called "rest"
* `color`: Adjusts the color of the bars; takes a string or a list of strings for the colors to use for the bars for each execution
* `bar_labels`: Adjusts whether labels are printed above the bars
* `figsize`: Takes a tuple of the size in inches to make the output figure

In [4]:
# Execute two-qubit Bell state again
sampler.options.default_shots = 1000

job = sampler.run([isa_circuit])
second_result = job.result()

# Plot results with custom options
plot_histogram(
    [
        result[0].data.meas.get_counts(),
        second_result[0].data.meas.get_counts(),
    ],
    legend=["first", "second"],
    sort="desc",
    figsize=(15, 12),
    color=["orange", "black"],
    bar_labels=False,
)

<Image src="/docs/images/guides/visualize-results/extracted-outputs/bd70e13f-5c52-42fb-8dde-980b15e3604a-0.avif" alt="Output of the previous code cell" />

## Plotting estimator results

Qiskit does not have a built-in function for plotting Estimator results, but you can use Matplotlib's [`bar` plot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html) for a quick visualization.

To demonstrate, the following cell estimates the expectation values of seven different observables on a quantum state.

In [5]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.transpiler import generate_preset_pass_manager
from matplotlib import pyplot as plt

# Simple estimation experiment to create results
qc = QuantumCircuit(2)
qc.h(0)
qc.crx(1.5, 0, 1)

observables_labels = ["ZZ", "XX", "YZ", "ZY", "XY", "XZ", "ZX"]
observables = [SparsePauliOp(label) for label in observables_labels]

service = QiskitRuntimeService()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)
isa_observables = [
    operator.apply_layout(isa_circuit.layout) for operator in observables
]

# Reshape observable array for broadcasting
reshaped_ops = np.fromiter(isa_observables, dtype=object)
reshaped_ops = reshaped_ops.reshape((7, 1))

estimator = Estimator(backend)
job = estimator.run([(isa_circuit, reshaped_ops)])
result = job.result()[0]
exp_val = job.result()[0].data.evs
print(result)

# Since the result array is structured as a 2D array where each element is a
# list containing a single value, you need to flatten the array.

# Plot using Matplotlib
plt.bar(observables_labels, exp_val.flatten())

PubResult(data=DataBin(evs=np.ndarray(<shape=(7, 1), dtype=float64>), stds=np.ndarray(<shape=(7, 1), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(7, 1), dtype=float64>), shape=(7, 1)), metadata={'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32})


<BarContainer object of 7 artists>

<Image src="/docs/images/guides/visualize-results/extracted-outputs/17c9893a-d1bf-4726-b444-6dce1d56805f-2.avif" alt="Output of the previous code cell" />

The following cell uses the estimated [standard error](https://en.wikipedia.org/wiki/Standard_error) of each result and adds them as error bars. See the [`bar` plot documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html) for a full description of the plot.

In [6]:
standard_error = job.result()[0].data.stds

_, ax = plt.subplots()
ax.bar(
    observables_labels,
    exp_val.flatten(),
    yerr=standard_error.flatten(),
    capsize=2,
)
ax.set_title("Expectation values (with standard errors)")

Text(0.5, 1.0, 'Expectation values (with standard errors)')

<Image src="/docs/images/guides/visualize-results/extracted-outputs/4eb79f4b-36b5-4797-a1a0-67d881d46ca4-1.avif" alt="Output of the previous code cell" />